<a href="https://colab.research.google.com/github/You2751/Colorization-of-gray-scale-images-using-WCGAN/blob/main/Image_Colorization_of_Gray_scale_Image_using_WCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-colorization:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F50512%2F125589%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240218%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240218T080127Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2d6243f2e6f39021e47f54208fde138b8bfecffcdc32000a5aa580a304404b49c3c1e734237b8b7a050549121373c5fbaba8d9d69bcbfbbe57fcc4b060ccaaa5611f24de6f214e2d5938912a2de127cb1bec3f18982f2ea48bbccbc341eb6344b9cc95d9480fe539c00838cf179307d7e0fb0ba8f081ad8fc37f6a342671d21fe99b89a088b67665b6a1809cf3bcf12aca16f6e968b08f70f85903f0a96c2ba6d3f1292acdafbe889c446037ee1418d2a5da8f30f0c860d182113437fce42f46bb803622eae4be21e3c9203413e7471451b7b8f07de8f81e6cd7aa96704f07a91fd22f9051d03d16b0acb4848f07455c4cf802360df96adcddb2ac99683e2a3d,pretrained-resunet-patchgan:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2788023%2F4899433%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240218%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240218T080127Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0d5a3e9ae48d8c0fa8dda73fd463062eabb341744034911bd3d5eaa155fab744671c9c683431f3bf8bafe40209e20f7ac154f69bf9137fb590c5027fa614d927bb07afb55de2fe488caa3e41df280f178aa4d92807db4c31f149bcbd9ea25acc1b1ba09b558bf17708026f62fbc31d40fbb61e303fb14a9233af26a94ca0efe14118b31d8a160031c5c3896349c8a0ae06f3b1f83468466ba183d4f357816601ce040de129ba559af4808d79376fcb014ed6fc0b8b0a1319643a9673b584ad932024ea9044b528c749142c4894587ab3ed9c32caf10d799af5aa3e1b92e3a5bb21d4232e01a8e52d0ea10e2d3e793d907a1e689d73a6d719e7d11a67180e5755'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install torchsummary
!pip install pytorch-lightning --no-cache-dir

In [ ]:
import os
from pathlib import Path
import glob
import gc
import time
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
from skimage.color import rgb2lab, lab2rgb
from scipy.stats import entropy

import pytorch_lightning as pl
from tqdm import tqdm
import warnings

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import transforms, models
from torchvision.models.inception import inception_v3
from torchsummary import summary

# Set matplotlib style
matplotlib.style.use('fivethirtyeight')

# Ignore unnecessary warnings
warnings.filterwarnings('ignore')

# Check and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



<h1><center>II. Data Preparation</center></h1>


In [ ]:
ab_path = "/kaggle/input/image-colorization/ab/ab/ab1.npy"
l_path = "/kaggle/input/image-colorization/l/gray_scale.npy"

# Load data and create dataset
L_df, ab_df = np.load(l_path)[:6000], np.load(ab_path)[:6000]
dataset = (L_df, ab_df)

# Perform garbage collection
gc.collect()


<h1><center>III. Data Exploration and Visualiaztion</center></h1>


In [ ]:
def lab_to_rgb(L, ab):
    L = L * 100
    ab = (ab - 0.5) * 128 * 2

    # Move tensors to the same device before concatenation
    L = L.to(ab.device)

    Lab = torch.cat([L, ab], dim=2).cpu().numpy()
    rgb_imgs = [lab2rgb(img) for img in Lab]
    return np.stack(rgb_imgs, axis=0)


In [ ]:
plt.figure(figsize=(30, 30))

for i in range(0, 15, 2):
    # Grayscale subplot
    grayscale_img = np.zeros((224, 224, 3))
    grayscale_img[:, :, 0] = L_df[i]
    plt.subplot(4, 4, i + 1)
    plt.title('Grayscale')
    plt.imshow(lab2rgb(grayscale_img))
    plt.axis("off")

    # Colored subplot
    colored_img = np.zeros((224, 224, 3))
    colored_img[:, :, 0] = L_df[i]  # Grayscale channel
    colored_img[:, :, 1:] = ab_df[i]  # Color channels
    colored_img = colored_img.astype('uint8')
    colored_img = cv2.cvtColor(colored_img, cv2.COLOR_LAB2RGB)
    plt.subplot(4, 4, i + 2)
    plt.title('Colored')
    plt.imshow(colored_img)
    plt.axis("off")

plt.show()


In [ ]:
gc.collect()


<h1><center>III. Data Loader</center></h1>


In [ ]:
class ImageColorizationDataset(Dataset):
    def __init__(self, L_data, ab_data, transform=None):
        """
        Black and White (L) Images and corresponding A&B Colors dataset.

        :param L_data: Grayscale images (L channel).
        :param ab_data: Color information (A&B channels).
        :param transform: Optional transform to be applied on samples.
        """
        self.L_data = L_data
        self.ab_data = ab_data
        self.transform = transform

    def __len__(self):
        return len(self.L_data)

    def __getitem__(self, idx):
        L = np.array(self.L_data[idx]).reshape((224, 224, 1))
        L = transforms.ToTensor()(L)

        ab = np.array(self.ab_data[idx])
        ab = transforms.ToTensor()(ab)

        return ab, L

# Batch size
batch_size = 1

# Prepare the Datasets
train_dataset = ImageColorizationDataset(L_data=L_df, ab_data=ab_df)
test_dataset = ImageColorizationDataset(L_data=L_df, ab_data=ab_df)

# Build DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)



<h1><center>IV. Data Modelling</center></h1>


#  Generator

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU(inplace=True)

        self.identity_map = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, inputs):
        x = inputs.clone().detach()

        out = self.relu1(self.bn1(self.conv1(x)))
        out = self.relu2(self.bn2(self.conv2(out)))

        residual = self.identity_map(x)
        skip = out + residual

        return self.relu(skip)


In [ ]:
class DownSampleConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DownSampleConv, self).__init__()

        self.maxpool = nn.MaxPool2d(2)
        self.resblock = ResBlock(in_channels, out_channels)

    def forward(self, inputs):
        return self.resblock(self.maxpool(inputs))


In [ ]:
class UpSampleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.res_block = ResBlock(in_channels + out_channels, out_channels)

    def forward(self, inputs, skip):
        x = self.upsample(inputs)
        x = torch.cat([x, skip], dim=1)
        x = self.res_block(x)
        return x

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_channel, output_channel, dropout_rate = 0.2):
        super().__init__()
        self.encoding_layer1_ = ResBlock(input_channel,64)
        self.encoding_layer2_ = DownSampleConv(64, 128)
        self.encoding_layer3_ = DownSampleConv(128, 256)
        self.bridge = DownSampleConv(256, 512)
        self.decoding_layer3_ = UpSampleConv(512, 256)
        self.decoding_layer2_ = UpSampleConv(256, 128)
        self.decoding_layer1_ = UpSampleConv(128, 64)
        self.output = nn.Conv2d(64, output_channel, kernel_size=1)
        self.dropout = nn.Dropout2d(dropout_rate)

    def forward(self, inputs):
        #Enocoder
        e1 = self.encoding_layer1_(inputs)
        e1 = self.dropout(e1)
        e2 = self.encoding_layer2_(e1)
        e2 = self.dropout(e2)
        e3 = self.encoding_layer3_(e2)
        e3 = self.dropout(e3)

        #Bridge
        bridge = self.bridge(e3)
        bridge = self.dropout(bridge)

        #Decoder
        d3 = self.decoding_layer3_(bridge, e3)
        d2 = self.decoding_layer2_(d3, e2)
        d1 = self.decoding_layer1_(d2, e1)

        #Output
        output = self.output(d1)
        return output

In [ ]:
model = Generator(1,2).to(device)
summary(model, (1, 224, 224), batch_size = 1)

# Discriminator ( Critic )

In [ ]:
class Critic(nn.Module):
    def __init__(self, in_channels=3):
        super(Critic, self).__init__()

        def critic_block(in_filters, out_filters, normalization=True):
            """Returns layers of each critic block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        blocks = [
            *critic_block(in_channels, 64, normalization=False),
            *critic_block(64, 128),
            *critic_block(128, 256),
            *critic_block(256, 512),
        ]

        self.model = nn.Sequential(
            *blocks,
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, 1)
        )

    def forward(self, ab, l):
        # Concatenate image and condition image by channels to produce input
        img_input = torch.cat((ab, l), 1)
        output = self.model(img_input)
        return output


In [ ]:
model = Critic(3).to(device)
summary(model, [(2, 224, 224), (1, 224, 224)], batch_size = 1)

# GAN

In [ ]:
def initialize_weights(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
        if hasattr(m, 'weight'):
            nn.init.normal_(m.weight, 0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            nn.init.constant_(m.bias, 0)


In [ ]:
def display_progress(cond, real, fake, current_epoch=0, figsize=(20, 15)):
    """
    Save cond, real (original), and generated (fake) images in one panel.
    """
    cond = cond.detach().cpu().permute(1, 2, 0)
    real = real.detach().cpu().permute(1, 2, 0)
    fake = fake.detach().cpu().permute(1, 2, 0)

    images = [cond, real, fake]
    titles = ['Input', 'Real', 'Generated']

    print(f'Epoch: {current_epoch}')
    fig, ax = plt.subplots(1, 3, figsize=figsize)

    for idx, (img, title) in enumerate(zip(images, titles)):
        if idx == 0:
            ab = torch.zeros((224, 224, 2))
            img = torch.cat([images[0] * 100, ab], dim=2).numpy()
            imgan = lab2rgb(img)
        else:
            imgan = lab_to_rgb(images[0], img)

        ax[idx].imshow(imgan)
        ax[idx].axis("off")
        ax[idx].set_title('{}'.format(title))

    plt.show()

In [ ]:
class CWGAN(pl.LightningModule):

    def __init__(self, in_channels, out_channels, learning_rate=0.0002, lambda_recon=100, display_step=10, lambda_gp=10, lambda_r1=10,):

        super().__init__()
        self.save_hyperparameters()

        self.display_step = display_step

        self.generator = Generator(in_channels, out_channels)
        self.critic = Critic(in_channels + out_channels)
        self.optimizer_G = optim.Adam(self.generator.parameters(), lr=learning_rate, betas=(0.5, 0.9))
        self.optimizer_C = optim.Adam(self.critic.parameters(), lr=learning_rate, betas=(0.5, 0.9))
        self.lambda_recon = lambda_recon
        self.lambda_gp = lambda_gp
        self.lambda_r1 = lambda_r1
        self.recon_criterion = nn.L1Loss()
        self.generator_losses, self.critic_losses  =[],[]

    def configure_optimizers(self):
        return [self.optimizer_C, self.optimizer_G]

    def generator_step(self, real_images, conditioned_images):
        # WGAN has only a reconstruction loss
        self.optimizer_G.zero_grad()
        fake_images = self.generator(conditioned_images)
        recon_loss = self.recon_criterion(fake_images, real_images)
        recon_loss.backward()
        self.optimizer_G.step()

        # Keep track of the average generator loss
        self.generator_losses += [recon_loss.item()]


    def critic_step(self, real_images, conditioned_images):
        self.optimizer_C.zero_grad()
        fake_images = self.generator(conditioned_images)
        fake_logits = self.critic(fake_images, conditioned_images)
        real_logits = self.critic(real_images, conditioned_images)

        # Compute the loss for the critic
        loss_C = real_logits.mean() - fake_logits.mean()

        # Compute the gradient penalty
        alpha = torch.rand(real_images.size(0), 1, 1, 1, requires_grad=True)
        alpha = alpha.to(device)
        interpolated = (alpha * real_images + (1 - alpha) * fake_images.detach()).requires_grad_(True)

        interpolated_logits = self.critic(interpolated, conditioned_images)

        grad_outputs = torch.ones_like(interpolated_logits, dtype=torch.float32, requires_grad=True)
        gradients = torch.autograd.grad(outputs=interpolated_logits, inputs=interpolated, grad_outputs=grad_outputs,create_graph=True, retain_graph=True)[0]


        gradients = gradients.view(len(gradients), -1)
        gradients_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
        loss_C += self.lambda_gp * gradients_penalty

        # Compute the R1 regularization loss
        r1_reg = gradients.pow(2).sum(1).mean()
        loss_C += self.lambda_r1 * r1_reg

        # Backpropagation
        loss_C.backward()
        self.optimizer_C.step()
        self.critic_losses += [loss_C.item()]

    def training_step(self, batch, batch_idx, optimizer_idx):
        real, condition = batch
        if optimizer_idx == 0:
            self.critic_step(real, condition)
        elif optimizer_idx == 1:
            self.generator_step(real, condition)
        gen_mean = sum(self.generator_losses[-self.display_step:]) / self.display_step
        crit_mean = sum(self.critic_losses[-self.display_step:]) / self.display_step
        if self.current_epoch%self.display_step==0 and batch_idx==0 and optimizer_idx==1:
            fake = self.generator(condition).detach()
            torch.save(cwgan.generator.state_dict(), "ResUnet_"+ str(self.current_epoch) +".pt")
            torch.save(cwgan.critic.state_dict(), "PatchGAN_"+ str(self.current_epoch) +".pt")
            print(f"Epoch {self.current_epoch} : Generator loss: {gen_mean}, Critic loss: {crit_mean}")
            display_progress(condition[0], real[0], fake[0], self.current_epoch)

In [ ]:
gc.collect()
cwgan = CWGAN(in_channels = 1, out_channels = 2 ,learning_rate=2e-4, lambda_recon=100, display_step=10)

In [ ]:
trainer = pl.Trainer(max_epochs=500, gpus=-1)
trainer.fit(cwgan, train_loader)


<h1><center>VI. Model Inferencing</center></h1>


In [ ]:
plt.figure(figsize=(30, 60))
idx = 1

for batch_idx, batch in enumerate(test_loader):
    real, condition = batch

    # Move data to the same device as the model
    real = real.to(device)
    condition = condition.to(device)

    # Forward pass through the generator
    pred = cwgan.generator(condition).detach().squeeze().permute(1, 2, 0)

    # Move data back to CPU for visualization if needed
    condition = condition.cpu().detach().squeeze(0).permute(1, 2, 0)
    real = real.cpu().detach().squeeze(0).permute(1, 2, 0)

    # Increase spacing between each picture
    plt.subplots_adjust(wspace=0, hspace=0.5)

    plt.subplot(6, 3, idx)
    plt.grid(False)

    ab = torch.zeros((224, 224, 2))
    img = torch.cat([condition * 100, ab], dim=2).numpy()
    imgan = lab2rgb(img)
    plt.imshow(imgan)
    plt.title('Input')

    plt.subplot(6, 3, idx + 1)
    ab = torch.zeros((224, 224, 2))
    imgan = lab_to_rgb(condition, real)
    plt.imshow(imgan)
    plt.title('Real')

    plt.subplot(6, 3, idx + 2)
    imgan = lab_to_rgb(condition, pred)
    plt.title('Generated')
    plt.imshow(imgan)

    idx += 3

    if idx >= 18:
        break



<h1><center>VI. Evaluation</center></h1>


## Inception Score

In [ ]:
with torch.no_grad():
    cwgan.generator.eval()
    all_preds = []
    all_real = []

    for batch_idx, (real, condition) in enumerate(test_loader):
        condition = condition.to(device)  # Move condition to the same device as the model
        pred = cwgan.generator(condition).detach()
        Lab = torch.cat([condition, pred], dim=1).cpu().numpy()  # Move to CPU before converting to numpy
        Lab_real = torch.cat([condition.cpu(), real.to(device).cpu()], dim=1).numpy()
        all_preds.append(Lab.squeeze())
        all_real.append(Lab_real.squeeze())

        if batch_idx == 500:
            break


In [ ]:
class InceptionScore:
    def __init__(self, device):
        self.device = device
        self.inception = self._initialize_inception()
        self.inception.eval()

    def _initialize_inception(self):
        inception = inception_v3(pretrained=True, transform_input=False).to(self.device)
        return inception

    def calculate_is(self, generated_images):
        generated_images = generated_images.to(self.device)

        with torch.no_grad():
            generated_features = self.inception(generated_images.view(-1, 3, 224, 224))

        generated_features = generated_features.view(generated_features.size(0), -1)
        p = F.softmax(generated_features, dim=1)

        kl = p * (torch.log(p) - torch.log(torch.tensor(1.0 / generated_features.size(1)).to(self.device)))
        kl = kl.sum(dim=1)

        return kl.mean().item(), kl.std().item()


In [ ]:
# Assuming all_preds and all_real are already PyTorch tensors
device = "cuda"  # or "cpu" if you don't have a GPU
is_calculator = InceptionScore(device)

# Ensure all_preds and all_real are PyTorch tensors
all_preds = torch.tensor(all_preds).float().to(device)
all_real = torch.tensor(all_real).float().to(device)

# Move tensors to CPU before converting to NumPy arrays
all_preds_cpu = all_preds.cpu().numpy()
all_real_cpu = all_real.cpu().numpy()

is_model = InceptionScore(device)

# Calculate the Inception Score
mean_real, std_real = is_model.calculate_is(all_real)
print("Inception Score of real images: mean: {:.4f}, std: {:.4f}".format(mean_real, std_real))
mean_is, std_is = is_model.calculate_is(all_preds)
print("Inception Score of fake images: mean: {:.4f}, std: {:.4f}".format(mean_is, std_is))


## Implemenation of Fréchet Inception Distance (FID)

In [ ]:
class FID:
    def __init__(self, device):
        self.device = device
        self.inception = inception_v3(pretrained=True, transform_input=False).to(self.device)
        self.inception.eval()
        self.mu = None
        self.sigma = None

    def preprocess_images(self, images):
        return images.view(-1, 3, 224, 224).to(self.device)

    def calculate_fid(self, real_images, generated_images):
        real_images, generated_images = self.preprocess_images(real_images), self.preprocess_images(generated_images)

        with torch.no_grad():
            real_features, generated_features = self.inception(real_images), self.inception(generated_images)

        real_features, generated_features = real_features.view(real_features.size(0), -1), generated_features.view(generated_features.size(0), -1)

        if self.mu is None:
            self.mu, self.sigma = real_features.mean(dim=0), real_features.std(dim=0)

        real_mu, real_sigma = real_features.mean(dim=0), real_features.std(dim=0)
        generated_mu, generated_sigma = generated_features.mean(dim=0), generated_features.std(dim=0)

        mu_diff, sigma_diff = real_mu - generated_mu, real_sigma - generated_sigma

        fid = mu_diff.pow(2).sum() + sigma_diff.pow(2).sum() + (self.sigma - generated_sigma).pow(2).sum()
        return fid.item()


In [ ]:
# Initialize the FID class
fid_calculator = FID(device)

# Calculate the FID
fid_value = fid_calculator.calculate_fid(all_real, all_preds)
print(f"FID: {fid_value:.4f}")
